In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO) 

C:\Users\bridge\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import pandas as pd

# run the bottom line if using bigtrain.csv and then delete unwanted columns
# df = pd.read_csv("bigtrain.csv")

In [3]:
# del df['eval_set']

In [4]:
# del df['add_to_cart_order']

In [7]:
# df.to_csv('train01.csv', encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv("train01.csv")

In [9]:
df.head(10)

,user_id,user_orders,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered,aisle_id,department_id
0,17,40,1737705,1,2,13,30,7350,0,115,7
1,17,40,1681401,2,5,10,3,7350,1,115,7
2,17,40,2680214,3,3,10,5,7350,1,115,7
3,17,40,3237467,5,6,17,5,7350,1,115,7
4,17,40,2616505,6,4,17,5,7350,1,115,7
5,17,40,2648319,7,1,13,4,7350,1,115,7
6,17,40,2430354,8,0,15,6,7350,1,115,7
7,17,40,900554,10,0,15,4,7350,1,115,7
8,17,40,805025,11,4,15,4,7350,1,115,7
9,17,40,912404,12,2,14,5,7350,1,115,7


In [10]:
df.order_id.nunique()

1629531

In [30]:
df.isnull().values.any()

False

In [23]:
user_orders = tf.feature_column.numeric_column('user_orders')
order_number = tf.feature_column.numeric_column('order_number')
days_since_prior_order = tf.feature_column.numeric_column('days_since_prior_order')

In [24]:
order_dow = tf.feature_column.categorical_column_with_vocabulary_list(
    'order_dow', ['0', '1', '2', '3', '4', '5', '6'])
order_id = tf.feature_column.categorical_column_with_hash_bucket(
    'order_id', hash_bucket_size=2000000)
user_id = tf.feature_column.categorical_column_with_hash_bucket(
    'user_id', hash_bucket_size=50000)
product_id = tf.feature_column.categorical_column_with_hash_bucket(
    'product_id', hash_bucket_size=50000)
order_hour_of_day = tf.feature_column.categorical_column_with_hash_bucket(
    'order_hour_of_day', hash_bucket_size=30)
aisle_id = tf.feature_column.categorical_column_with_hash_bucket(
    'aisle_id', hash_bucket_size=100)
department_id = tf.feature_column.categorical_column_with_hash_bucket(
    'department_id', hash_bucket_size=100)

In [25]:
wide_columns = [user_id, user_orders, order_id, order_number,
      order_dow, order_hour_of_day, days_since_prior_order, product_id,
      aisle_id, department_id]

In [26]:
deep_columns = [
    # Multi-hot indicator columns for columns with fewer possibilities
    tf.feature_column.indicator_column(aisle_id),
    tf.feature_column.indicator_column(department_id),
    tf.feature_column.indicator_column(order_dow),
    tf.feature_column.indicator_column(order_hour_of_day),
    # Embeddings for categories with more possibilities
    tf.feature_column.embedding_column(user_id, dimension=20),
    tf.feature_column.embedding_column(product_id, dimension=20),
    tf.feature_column.embedding_column(order_id, dimension=20),
    # Numerical columns
    order_number,
    user_orders,
    days_since_prior_order
]


In [27]:
model = tf.estimator.DNNLinearCombinedClassifier(
    model_dir='/RecSys/attempt1',
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[100, 50])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/RecSys/attempt1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000000013953278>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
BATCH_SIZE = 40

def generate_input_fn(filename, num_epochs=None, shuffle=True, batch_size=BATCH_SIZE):
    df = pd.read_csv(filename, header=None, names=COLUMNS)
    labels = df["reordered"]
        
    return tf.estimator.inputs.pandas_input_fn(
        x=df,
        y=labels,
        batch_size=batch_size,
        num_epochs=num_epochs,
        shuffle=shuffle)

print('input function configured')

input function configured


In [32]:
COLUMNS = ["user_id", "user_orders", "order_id", "order_number", "order_dow", "order_hour_of_day", 
           "days_since_prior_order", "product_id", "reordered", "aisle_id", "department_id"]

In [33]:
%%time 

train_file = str("train01.csv") 

model.train(input_fn=generate_input_fn(train_file), 
      steps=1000)

print('training done')

C:\Users\bridge\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InternalError'>, Unable to get element as bytes.


InternalError: Unable to get element as bytes.